# Optimize Interface Film Position in Gr/Ni(111) Interface

## 0. Introduction

This notebook demonstrates how to create Gr/Ni(111) interface and optimize the film position.

Following the manuscript:

> **Arjun Dahal, Matthias Batzill**
> "Graphene–nickel interfaces: a review"
> Nanoscale, 6(5), 2548. (2014)
> [DOI: 10.1039/c3nr05279f](https://doi.org/10.1039/c3nr05279f)
 
Recreating interface and shifting the film to the most favorable energy position showed in the image below. Fig 1. b.

<img src="https://github.com/Exabyte-io/documentation/raw/12617167278ae3523adc028583b21ea4e8ebd197/images/tutorials/materials/optimization/optimization_interface_film_xy_position_graphene_nickel/0-figure-from-manuscript.webp" alt="Optimal position of the film" width="400"/>

## 1. Prepare the Environment
### 1.1. Install Packages


In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("specific_examples|create_interface_with_min_strain_zsl.ipynb")


### 1.2. Set Parameters


In [ ]:
# Material selection
SUBSTRATE_NAME = "Nickel"
FILM_NAME = "Graphene"

# Slab parameters
FILM_MILLER_INDICES = (0, 0, 1)
FILM_THICKNESS = 1  # in atomic layers
FILM_VACUUM = 0.0  # in angstroms
FILM_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
FILM_USE_ORTHOGONAL_Z = True

SUBSTRATE_MILLER_INDICES = (1, 1, 1)
SUBSTRATE_THICKNESS = 4  # in atomic layers
SUBSTRATE_VACUUM = 0.0  # in angstroms
SUBSTRATE_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
SUBSTRATE_USE_ORTHOGONAL_Z = True

# Interface parameters
MAX_AREA = 50  # in Angstrom^2
INTERFACE_DISTANCE = 2.58  # in Angstrom
INTERFACE_VACUUM = 20.0  # in Angstrom

# Optimization parameters
GRID_SIZE = (20, 20)  # Resolution of the x-y grid
GRID_RANGE_X = (-0.5, 0.5)  # Range to search in x direction
GRID_RANGE_Y = (-0.5, 0.5)  # Range to search in y direction
USE_CARTESIAN = False  # Whether to use Cartesian coordinates

# Visualization parameters
SHOW_3D_LANDSCAPE = False  # Whether to show 3D energy landscape
STRUCTURE_REPETITIONS = [3, 3, 1]  # Repetitions for structure visualization


## 2. Create Interface Material
### 2.1. Load Materials


In [ ]:
from utils.visualize import visualize_materials
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material

substrate = Material(Materials.get_by_name_first_match(SUBSTRATE_NAME))
film = Material(Materials.get_by_name_first_match(FILM_NAME))

# Preview materials
visualize_materials([substrate, film], repetitions=STRUCTURE_REPETITIONS, rotation="0x")


### 2.2. Create Slabs

In [ ]:
from mat3ra.made.tools.build.slab import SlabConfiguration, PymatgenSlabGeneratorParameters, get_terminations, \
    create_slab

# Configure slabs
film_slab_configuration = SlabConfiguration(
    bulk=film,
    miller_indices=FILM_MILLER_INDICES,
    thickness=FILM_THICKNESS,
    vacuum=FILM_VACUUM,
    xy_supercell_matrix=FILM_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=FILM_USE_ORTHOGONAL_Z
)

substrate_slab_configuration = SlabConfiguration(
    bulk=substrate,
    miller_indices=SUBSTRATE_MILLER_INDICES,
    thickness=SUBSTRATE_THICKNESS,
    vacuum=SUBSTRATE_VACUUM,
    xy_supercell_matrix=SUBSTRATE_XY_SUPERCELL_MATRIX,
    use_orthogonal_z=SUBSTRATE_USE_ORTHOGONAL_Z,
)

# Get terminations
params = PymatgenSlabGeneratorParameters(symmetrize=False)
film_slab_terminations = get_terminations(film_slab_configuration, params)
substrate_slab_terminations = get_terminations(substrate_slab_configuration, params)

# Create slabs
film_slabs = [create_slab(film_slab_configuration, termination) for termination in film_slab_terminations]
substrate_slabs = [create_slab(substrate_slab_configuration, termination, params) for termination in
                   substrate_slab_terminations]

# Visualize slabs
visualize_materials([{"material": slab, "title": slab.metadata["build"]["termination"]} for slab in film_slabs],
                    repetitions=STRUCTURE_REPETITIONS, rotation="-90x")
visualize_materials([{"material": slab, "title": slab.metadata["build"]["termination"]} for slab in substrate_slabs],
                    repetitions=STRUCTURE_REPETITIONS, rotation="-90x")


### 2.3. Create Interface


In [ ]:
from mat3ra.made.tools.build.interface import InterfaceConfiguration, ZSLStrainMatchingParameters, \
    ZSLStrainMatchingInterfaceBuilderParameters, ZSLStrainMatchingInterfaceBuilder
from itertools import product

# Get termination pairs
termination_pairs = list(product(film_slab_terminations, substrate_slab_terminations))
print("Termination Pairs (Film, Substrate):")
for idx, termination_pair in enumerate(termination_pairs):
    print(f"    {idx}: {termination_pair}")

# Create interface with first termination pair
termination_pair = termination_pairs[0]
film_termination, substrate_termination = termination_pair

interface_configuration = InterfaceConfiguration(
    film_configuration=film_slab_configuration,
    substrate_configuration=substrate_slab_configuration,
    film_termination=film_termination,
    substrate_termination=substrate_termination,
    distance_z=INTERFACE_DISTANCE,
    vacuum=INTERFACE_VACUUM
)

# Build interface using ZSL matching
zsl_params = ZSLStrainMatchingParameters(max_area=MAX_AREA)
builder = ZSLStrainMatchingInterfaceBuilder(
    build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
        strain_matching_parameters=zsl_params
    )
)

interfaces = builder.get_materials(configuration=interface_configuration)
interface_material = interfaces[0]  # Select first interface
interface_material.name = f"{FILM_NAME}_{SUBSTRATE_NAME}_interface"

# Visualize interface
visualize_materials([interface_material], repetitions=STRUCTURE_REPETITIONS)
visualize_materials([interface_material], repetitions=STRUCTURE_REPETITIONS, rotation="-90x")


## 3. Optimize Interface Position
### 3.1. Calculate Energy Landscape


In [ ]:
from mat3ra.made.tools.build.interface import get_optimal_film_displacement
from mat3ra.made.tools.modify import interface_displace_part
from mat3ra.made.tools.optimize import evaluate_calculator_on_xy_grid
from mat3ra.made.tools.calculate.calculators import InterfaceMaterialCalculator

calculator = InterfaceMaterialCalculator()

# Calculate energy landscape
xy_matrix, energy_matrix = evaluate_calculator_on_xy_grid(
    material=interface_material,
    calculator_function=calculator.get_energy,
    modifier=interface_displace_part,
    grid_size_xy=GRID_SIZE,
    grid_range_x=GRID_RANGE_X,
    grid_range_y=GRID_RANGE_Y,
    use_cartesian_coordinates=USE_CARTESIAN
)

# Find optimal position
optimal_displacement = get_optimal_film_displacement(
    material=interface_material,
    calculator=calculator,
    grid_size_xy=GRID_SIZE,
    grid_range_x=GRID_RANGE_X,
    grid_range_y=GRID_RANGE_Y,
    use_cartesian_coordinates=USE_CARTESIAN
)
print(f"\nOptimal displacement vector: {optimal_displacement}")


### 3.2. Visualize Energy Landscape


In [ ]:
from mat3ra.utils.jupyterlite.plot import plot_2d_heatmap, plot_3d_surface

x_values, y_values = xy_matrix
# Plot energy landscape
plot_2d_heatmap(x_values, y_values, energy_matrix,
                optimal_point=optimal_displacement[:2],
                title="Energy Heatmap",
                labels={"x": "x", "y": "y", "z": "Energy"})
if SHOW_3D_LANDSCAPE:
    plot_3d_surface(x_values, y_values, energy_matrix,
                    optimal_point=optimal_displacement[:2],
                    title="Energy Landscape",
                    labels={"x": "x", "y": "y", "z": "Energy"})

# Create and visualize optimized material
optimized_material = interface_displace_part(
    interface_material,
    displacement=optimal_displacement,
    use_cartesian_coordinates=USE_CARTESIAN
)

### 3.3. Visualize Material


In [ ]:
print("\nVisualization of original and optimized materials:")
visualize_materials([{"material": interface_material, "title": "Original Interface"},
                     {"material": optimized_material, "title": "Optimized Interface"}],
                    repetitions=STRUCTURE_REPETITIONS,
                    )
visualize_materials([{"material": interface_material, "title": "Original Interface"},
                     {"material": optimized_material, "title": "Optimized Interface"}],
                    repetitions=STRUCTURE_REPETITIONS,
                    rotation="-90x")

# 4. Save optimized material

In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials

optimized_material.name = f"{interface_material.name}_optimized_xy"
set_materials(optimized_material)
download_content_to_file(optimized_material, f"{interface_material.name}_optimized_xy.json")